<a href="https://colab.research.google.com/github/GnuAkitoru/ChordCreateSystem/blob/main/ChordCreateSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

インストール


In [1]:
import gspread
import math

google スプレッドシートへのアクセスを許可する

In [2]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

スプレッドシートからデータを持ってきて、

(メロディー,コード)の組でタプルにまとめ、それらをdatalstに入れる

In [4]:
#スプレッドシートのファイルとページの指定

ss_name = "コード進行生成用DB"
workbook = gc.open(ss_name)
worksheet = workbook.worksheet("本DB")

datalst = [] #本システムで使用するデータを入れるリスト

MC = [] #リスト作成用の仮リスト
datalstp = [] #これもリスト作成用の仮リスト

#メロディーとコードを数字に変換するための表(プログラム内で扱いやすいように)
ChangeM = ['c', 'd', 'e', 'f', 'g', 'a', 'b']
ChangeC = ['C', 'Dm', 'Em', 'F', 'G', 'Am']

#メロディーとコードをリストにまとめる
cell_list = worksheet.range("A1:B190")
for cell in cell_list:
  MC.append(cell.value)
print(MC) #[メロディー,コード,メロディー,コード,...]

#MC内の文字を数字に変換する
for i in range(len(MC)):
  if i % 2 == 0:
    henkan = ChangeM.index(MC[i])
    datalstp.append(henkan)
  elif i % 2 == 1:
    henkan = ChangeC.index(MC[i])
    datalstp.append(henkan)
print(datalstp)

#(メロディー,コード)の組のタプルを作り、datalstに追加する
for j in range((len(datalstp) // 2)):
  TEIO = (datalstp[2 * j], datalstp[2 * j  + 1])
  datalst.append(TEIO)

print('datalst↓')
print(datalst)

['c', 'C', 'e', 'C', 'e', 'C', 'e', 'C', 'd', 'G', 'f', 'G', 'f', 'G', 'f', 'G', 'e', 'C', 'g', 'C', 'g', 'C', 'g', 'C', 'f', 'Dm', 'a', 'Dm', 'a', 'Dm', 'a', 'Dm', 'g', 'C', 'd', 'C', 'a', 'F', 'a', 'F', 'a', 'Dm', 'e', 'Dm', 'b', 'G', 'b', 'G', 'b', 'Em', 'f', 'Em', 'c', 'Am', 'c', 'Am', 'a', 'F', 'b', 'G', 'c', 'C', 'c', 'C', 'c', 'C', 'e', 'C', 'e', 'F', 'e', 'F', 'e', 'G', 'g', 'G', 'e', 'C', 'g', 'C', 'g', 'C', 'e', 'C', 'g', 'Em', 'g', 'Em', 'a', 'Am', 'e', 'Am', 'd', 'G', 'e', 'C', 'a', 'F', 'b', 'F', 'a', 'Em', 'c', 'Am', 'd', 'F', 'd', 'F', 'g', 'G', 'e', 'G', 'c', 'C', 'b', 'C', 'a', 'C', 'g', 'C', 'b', 'Em', 'b', 'Em', 'd', 'Em', 'e', 'Em', 'a', 'F', 'a', 'F', 'g', 'F', 'c', 'F', 'g', 'G', 'd', 'G', 'c', 'F', 'c', 'F', 'd', 'Em', 'g', 'Em', 'd', 'Am', 'c', 'Am', 'c', 'C', 'c', 'C', 'c', 'F', 'c', 'F', 'd', 'Em', 'c', 'Em', 'a', 'Am', 'g', 'Am', 'g', 'C', 'g', 'C', 'c', 'F', 'e', 'F', 'g', 'G', 'c', 'G', 'g', 'Em', 'g', 'Em', 'g', 'Am', 'g', 'Am', 'c', 'F', 'e', 'F', 'g', 'G

In [5]:
#コードメロディー間合致確率の計算

lstBc = []
lstBdm = []
lstBem = []
lstBf = []
lstBg = []
lstBam = []
lstB = [lstBc, lstBdm, lstBem, lstBf, lstBg, lstBam]

KRc = []     #確率を計算するため、データを振り分けて保存するリスト
KRdm = []    #（すべての確率計算で使われるリスト。その都度計算前にリセットします)
KRem = []
KRf = []
KRg = []
KRam = []
KRlst = [KRc, KRdm, KRem, KRf, KRg, KRam]

for i in range(len(datalst)):
  melo = datalst[i][0]
  chord = datalst[i][1]
  KRlst[chord].append(melo)
print(KRlst)

for j in range(6):       #確率を求めてリストにする(例：lstB[0][2]　-> コード:Cがメロディ:ミに合う確率)
  for k in range(7):
    if len(KRlst[j]) == 0:
      lstB[j] = [0,0,0,0,0,0]  #母数がそもそも０の場合は、もう確率は全部０
    else:
      lstBapp = KRlst[j].count(k) / len(KRlst[j])
      lstB[j].append(lstBapp)

print('lstB↓')
print(lstB)



[[0, 2, 2, 2, 2, 4, 4, 4, 4, 1, 0, 0, 0, 2, 2, 4, 4, 2, 2, 0, 6, 5, 4, 0, 0, 4, 4, 1, 1, 0, 0, 0, 0, 2, 2, 0, 2, 2, 4, 0, 2, 4, 0, 4, 0, 2, 4, 2, 0, 0, 1, 2, 1, 4, 1, 2, 4, 2], [3, 5, 5, 5, 5, 2], [6, 3, 4, 4, 5, 6, 6, 1, 2, 1, 4, 1, 0, 4, 4, 0, 0, 6, 5, 4, 0, 0, 4, 0, 2], [5, 5, 5, 2, 2, 5, 6, 1, 1, 5, 5, 4, 0, 0, 0, 0, 0, 0, 2, 0, 2, 5, 5, 5, 0, 0, 6, 3, 5, 3, 1, 3, 3, 1, 5, 5, 3, 5], [1, 3, 3, 3, 6, 6, 6, 2, 4, 1, 4, 2, 4, 1, 4, 0, 4, 2, 1, 4, 3, 1, 1, 1, 4, 4, 2, 1, 2, 1, 1, 6, 6, 4, 5, 1, 1, 1, 4], [0, 0, 5, 2, 0, 1, 0, 5, 4, 4, 4, 2, 2, 2, 0, 0, 0, 5, 3, 5, 1, 2, 1, 2]]
lstB↓
[[0.29310344827586204, 0.10344827586206896, 0.3103448275862069, 0.0, 0.25862068965517243, 0.017241379310344827, 0.017241379310344827], [0.0, 0.0, 0.16666666666666666, 0.16666666666666666, 0.0, 0.6666666666666666, 0.0], [0.24, 0.12, 0.08, 0.04, 0.28, 0.08, 0.16], [0.23684210526315788, 0.10526315789473684, 0.10526315789473684, 0.13157894736842105, 0.02631578947368421, 0.34210526315789475, 0.05263157894736842],

In [6]:
#コード間遷移確率の計算

lst12c = []
lst12dm = []
lst12em = []
lst12f = []
lst12g = []
lst12am = []
lst12 = [lst12c, lst12dm, lst12em, lst12f, lst12g, lst12am]

KRc = []
KRdm = []
KRem = []
KRf = []
KRg = []
KRam = []
KRlst = [KRc, KRdm, KRem, KRf, KRg, KRam] #lstBが終わったので初期化

for i in range(len(datalst) - 1):
  chord = datalst[i][1]
  nextchord = datalst[i + 1][1]
  if chord == nextchord:    #コードとその次のコードが同じだった場合、「移り変わり」ではないので、確率を求めるためのリストを入れない
    pass
  else:
    KRlst[chord].append(nextchord)

print(KRlst)

for j in range(6):       #確率を求めてリストにする　　(例:lst12[0][3] -> CからFへの遷移確率)
  for k in range(6):
    if len(KRlst[j]) == 0:
      lst12[j] = [0,0,0,0,0,0]  #母数がそもそも０の場合は、もう確率は全部０
    else:
      lst12app = KRlst[j].count(k) / len(KRlst[j])
      lst12[j].append(lst12app)

print('lst12変更前↓')
print(lst12)

[[4, 1, 3, 3, 2, 3, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 4, 4, 2, 2, 4, 3, 4], [0, 4], [5, 5, 5, 3, 5, 5, 5, 0, 3, 3, 5, 3, 0], [1, 4, 4, 2, 4, 4, 2, 2, 4, 4, 0, 0, 4, 0, 4, 5, 4, 4, 0, 0, 4], [0, 2, 0, 0, 0, 0, 3, 2, 0, 0, 5, 0, 0, 5, 5, 0, 0, 3, 0, 0, 5, 5], [3, 4, 3, 0, 0, 3, 4, 2, 2, 2, 2, 3, 3]]
lst12変更前↓
[[0.0, 0.041666666666666664, 0.16666666666666666, 0.4166666666666667, 0.375, 0.0], [0.5, 0.0, 0.0, 0.0, 0.5, 0.0], [0.15384615384615385, 0.0, 0.0, 0.3076923076923077, 0.0, 0.5384615384615384], [0.23809523809523808, 0.047619047619047616, 0.14285714285714285, 0.0, 0.5238095238095238, 0.047619047619047616], [0.5909090909090909, 0.0, 0.09090909090909091, 0.09090909090909091, 0.0, 0.22727272727272727], [0.15384615384615385, 0.0, 0.3076923076923077, 0.38461538461538464, 0.15384615384615385, 0.0]]


In [9]:
#冒頭コード、末尾コードは手計算で確率を出しました。

lst01 = [0.4, 0, 0, 0.3, 0.0, 0.3] #始まりのコードになる確率[C,Dm,Em,F,G,Am]
lst45 = [0.4, 0, 0, 0.1, 0.2, 0.2] #終わりのコードになる確率[C,Dm,Em,F,G,Am]

In [ ]:
print('lst12変更前↓')
print(lst12)

k = 0.15 #ドミナントなどの移り変わりに基づき、確率を底上げするための変数 (主に足りないなと思った部分に追加する)
lst12[0][5] += (k / 2) #C-Am
lst12[3][2] += k #F-Em
lst12[4][2] += k #G-Em
lst12[4][5] += k #G-Am
lst12[5][1] += (k / 2) #Am-Dm

print('lst12変更後↓')
print(lst12)

In [ ]:
#log使用　入力：c,d,e,f,g,a,b
user_input = input()
melo = []

ChangeM = ['c', 'd', 'e', 'f', 'g', 'a', 'b']
#入力されたメロディを数字に変換する。
for i in range(len(user_input)):
  henkanm = ChangeM.index(user_input[i])
  melo.append(henkanm)
#user_input = cde -> melo = [0,1,2]

output = []
Keisan = []
for i in range(len(melo)):
  Keisan.append(melo[(len(melo) - 1) - i])
#計算する順番に合わせるため、リストの順序を逆さまにする

chord = []
table = {}
afterchord = -1 #afterchordやtableでは、0=C 1=Dm 2=Em 3=F 4=G 5=Am
nowmelo = -1 #nowmeloでは、0＝ド,1＝レ,2＝ミ,3＝ファ,4＝ソ,5＝ラ,6＝シ
RuiKak = 1
logRuiKak = 0

def zelog(x):
  if x == 0:
    return -1000
  else:
    return math.log10(x)

#浮動小数点数を乗算する代わりにmath.log10()を使ってみようと思ったが、確率が０の場合、log10(0)を定義できない
#なので一旦、log10(0)を-1000として返すzelog(x)を定義。入力が長くない限り、０でない計算値が10の-1000乗を下回ることはない
#これは、確率が1番高いもののみを選ぶ時有効だと思われる。確率が本来０のものの中で、数値の大小が生まれているため。

for i in range(len(Keisan)):
  nowmelo = Keisan[i] #いまコードを決めているメロディー
  if i == 0:     #一番最後のコード
    for j in range(6):
      table[j] = zelog(lst45[j]) + zelog(lstB[j][nowmelo])
  elif i == len(Keisan) - 1:   #一番最初のコード
    for j in range(6):
      table[j] = zelog(lst01[j]) + zelog(lstB[j][nowmelo]) + zelog(lst12[j][afterchord]) + logRuiKak
  else:   #途中のコード
    for j in range(6):
      table[j] = zelog(lst12[j][afterchord]) + zelog(lstB[j][nowmelo]) + logRuiKak
  sortable = sorted(table.items(), key=lambda x:x[1], reverse=True) #sortable
  afterchord = sortable[0][0]
  logRuiKak = logRuiKak + sortable[0][1]
  henkanc = ChangeC[sortable[0][0]]#ChangeC = ['C', 'Dm', 'Em', 'F', 'G', 'Am']
  output.insert(0 , henkanc) #数字をコード名に変え、リストの0番目に挿入

print(output)


cgggcgddcgggcbcc
['F', 'C', 'G', 'C', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Em', 'Am', 'Em', 'F', 'C']


In [10]:
#zelog使用　入力：c,d,e,f,g,a,b
user_input = input()
melo = [] #入力は ドレミファソラシド を用いる

ChangeM = ['ド', 'レ', 'ミ', 'フ', 'ソ', 'ラ', 'シ']

for i in range(len(user_input)):
  if user_input[i] == 'ァ':
    pass
  else:
    henkanm = ChangeM.index(user_input[i])
    melo.append(henkanm)

#print(melo) #user_input = ドレミ -> melo = [0,1,2]

#melo = [0,1,2,3,5,1,0] #入力
output = []
Keisan = []
for i in range(len(melo)):
  Keisan.append(melo[(len(melo) - 1) - i])
#計算する順番に合わせるため、リストの順序を逆さまにする

chord = []
table = {}
afterchord = -1 #afterchordやtableでは、0=C 1=Dm 2=Em 3=F 4=G 5=Am
nowmelo = -1 #nowmeloでは、0＝ド,1＝レ,2＝ミ,3＝ファ,4＝ソ,5＝ラ,6＝シ
RuiKak = 1
logRuiKak = 0

def zelog(x):
  if x == 0:
    return -1000
  else:
    return math.log10(x)

for i in range(len(Keisan)):
  nowmelo = Keisan[i] #いまコードを決めているメロディー
  if i == 0:     #一番最後のコード
    for j in range(6):
      table[j] = zelog(lst45[j]) + zelog(lstB[j][nowmelo])
  elif i == len(Keisan) - 1:   #一番最初のコード
    for j in range(6):
      table[j] = zelog(lst01[j]) + zelog(lstB[j][nowmelo]) + zelog(lst12[j][afterchord]) + logRuiKak
  else:   #途中のコード
    for j in range(6):
      table[j] = zelog(lst12[j][afterchord]) + zelog(lstB[j][nowmelo]) + logRuiKak
  sortable = sorted(table.items(), key=lambda x:x[1], reverse=True)
  afterchord = sortable[0][0]
  logRuiKak = logRuiKak + sortable[0][1]
  henkanc = ChangeC[sortable[0][0]]#ChangeC = ['C', 'Dm', 'Em', 'F', 'G', 'Am']
  output.insert(0 , henkanc) #数字をコード名に変え、リストの0番目に挿入

print(output)


ドソソソドソレレドソソソドシドド
['F', 'C', 'G', 'C', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Em', 'Am', 'Em', 'F', 'C']


##ここから下はシステム制作の際に使ったノート、メモです。

# 何やっているか
「melo」リストにメロディーを入力します。

処理の順番は最後から最初なので meloを逆さまにし、「Keisan」リストに入れます。

# コードの処理
「for i in range(len(Keisan)):」から、メロディーの処理を始めます。


辞書の「table」に、key=[コード名],item=[メロディーとそのコードが合う確率]として
C,Dm,Em,F,G,Amの情報が追加されます。
その辞書を値でソートすると、0番目に一番確率の高いものが来るので、
afterchode（計算されたコードを保存する）にコード名が代入され、
RuiKak（累積確率）と確率を乗算します。

一番最後のコード：一番最後になる確率　* メロディーと合う確率

一番最初のコード：一番最初になる確率　* メロディーと合う確率 * afterchodeに移る確率 * 累積確率

途中のコード：　afterchodeに移る確率　*メロディーと合う確率 * 累積確率




In [ ]:
#コード遷移確率を、テンプレコード進行で計算してみる

KRc = []
KRdm = []
KRem = []
KRf = []
KRg = []
KRam = []
KRlst = [KRc, KRdm, KRem, KRf, KRg, KRam]  #初期化

chordlst = [] #持ってきたデータを数字に変換して保存するリスト
ChangeC = ['C', 'Dm', 'Em', 'F', 'G', 'Am']
lst12c = [0,0,0,0,0,0]
lst12dm = [0,0,0,0,0,0]
lst12em = [0,0,0,0,0,0]
lst12f = [0,0,0,0,0,0]
lst12g = [0,0,0,0,0,0]
lst12am = [0,0,0,0,0,0]  #初期状態として確率すべて０にしておく。
lst12 = [lst12c, lst12dm, lst12em, lst12f, lst12g, lst12am]

worksheet = workbook.worksheet("テンプレコード集")
data_temp = []


for n in range(7): #カノン 1451進行以外
  chordlst = []
  min = 5*n + 1
  max = 5*n + 5
  cell_list = worksheet.range("A"+str(min)+":A"+str(max)) #データを5セル分もってくる
  for cell in cell_list:
    henkanx = cell.value
    Mcqueen = ChangeC.index(cell.value)
    chordlst.append(Mcqueen) #例F G Em Am Fを[3, 4, 2, 5, 3]に変換
  for i in range(len(chordlst) - 1):
    chord = chordlst[i]
    nextchord = chordlst[i + 1]
    KRlst[chord].append(nextchord) #「コードとその次のコードが同じ」がないようなデータにしています　#例[[], [], [5], [4], [2], [3]]
  for j in range(6):       #確率を求めてリストにする　　(例:lst12[0][3] -> CからFへの遷移確率)
    for k in range(6):
      if len(KRlst[j]) == 0:
        lst12[j] = [0,0,0,0,0,0]  #母数がそもそも０の場合は、もう確率は全部０
      else:
        lst12app = KRlst[j].count(k) / len(KRlst[j])
        lst12[j][k] = (lst12app / (n+1)) + (lst12[j][k] * n / (n+1) )    #処理時点で登録されている確率にまぜる
print(lst12)

In [ ]:
#カノン進行と1451

chordlst = []
cell_list = worksheet.range("C1:C9") #カノン進行
for cell in cell_list:
  henkanx = cell.value
  Mcqueen = ChangeC.index(cell.value)
  chordlst.append(Mcqueen) #例F G Em Am Fを[3, 4, 2, 5, 3]に変換
for i in range(len(chordlst) - 1):
  chord = chordlst[i]
  nextchord = chordlst[i + 1]
  KRlst[chord].append(nextchord) #「コードとその次のコードが同じ」がないようなデータにしています　#例[[], [], [5], [4], [2], [3]]
for j in range(6):       #確率を求めてリストにする　　(例:lst12[0][3] -> CからFへの遷移確率)
  for k in range(6):
    if len(KRlst[j]) == 0:
      lst12[j] = [0,0,0,0,0,0]  #母数がそもそも０の場合は、もう確率は全部０
    else:
      lst12app = KRlst[j].count(k) / len(KRlst[j])
      lst12[j][k] = lst12[j][k] = (lst12app / (9)) + (lst12[j][k] * 8 / 9 )    #処理時点で登録されている確率との平均をとる 9個目

print(lst12)

chordlst = []
cell_list = worksheet.range("C10:C13") #1451進行
for cell in cell_list:
  henkanx = cell.value
  Mcqueen = ChangeC.index(cell.value)
  chordlst.append(Mcqueen) #例F G Em Am Fを[3, 4, 2, 5, 3]に変換
for i in range(len(chordlst) - 1):
  chord = chordlst[i]
  nextchord = chordlst[i + 1]
  KRlst[chord].append(nextchord) #「コードとその次のコードが同じ」がないようなデータにしています　#例[[], [], [5], [4], [2], [3]]
for j in range(6):       #確率を求めてリストにする　　(例:lst12[0][3] -> CからFへの遷移確率)
  for k in range(6):
    if len(KRlst[j]) == 0:
      lst12[j] = [0,0,0,0,0,0]  #母数がそもそも０の場合は、もう確率は全部０
    else:
      lst12app = KRlst[j].count(k) / len(KRlst[j])
      lst12[j][k] = (lst12app + lst12[j][k]) / 2    #処理時点で登録されている確率との平均をとる


print(lst12)

In [ ]:
#コード遷移確率を、テンプレコード進行で計算してみるのメモ
data_temp = []
chordlst = []

cell_list = worksheet.range(A1:A5")
for cell in cell_list:
  data_temp.append(cell.value)
print(data_temp) #テンプレコード4+1個分をまとめる

for i in range(len(data_temp)):
  Mcqueen = ChangeC.index(data_temp[i])
  chordlst.append(Mcqueen)

print(chordlst)
print('↑と↓はどちらも「データを持ってきてデータを数字に変換」している')

data_temp = []
chordlst = []

cell_list = worksheet.range("A1:A5")
for cell in cell_list:
  henkanx = cell.value
  Mcqueen = ChangeC.index(cell.value)
  chordlst.append(Mcqueen)

print(chordlst)

In [ ]:
#zelog未使用
user_input = input()
melo = [] #入力は c,d,e,f,g,a,b を用いる

ChangeM = ['c', 'd', 'e', 'f', 'g', 'a', 'b']
#入力されたメロディを数字に変換する。
for i in range(len(user_input)):
  henkanm = ChangeM.index(user_input[i])
  melo.append(henkanm)

print(melo) #user_input = cde -> melo = [0,1,2]

#melo = [0,1,2,3,5,1,0] #入力
output = []

Keisan = []
for i in range(len(melo)):
  Keisan.append(melo[(len(melo) - 1) - i])
#計算する順番に合わせるため、リストの順序を逆さまにする

chode = []
table = {}
afterchode = -1 #afterchodeやtableでは、0=C 1=Dm 2=Em 3=F 4=G 5=Am
nowmelo = -1 #nowmeloでは、0＝ド,1＝レ,2＝ミ,3＝ファ,4＝ソ,5＝ラ,6＝シ
RuiKak = 1
#logRuiKak = 0

for i in range(len(Keisan)): #入力(実際は逆さまになったリスト)のメロディの個数分繰り返す
  nowmelo = Keisan[i] #いまコードを決めているメロディーをnowmeloに代入
  if i == 0:     #一番最後のコードを決める
    for j in range(6):
      table[j] = lst45[j] * lstB[j][nowmelo]
  elif i == len(Keisan) - 1:   #一番最初のコード
    for j in range(6):
      table[j] = lst01[j] * lstB[j][nowmelo] * lst12[j][afterchode] * RuiKak
  else:   #途中のコード
    for j in range(6):
      table[j] = lst12[j][afterchode] * lstB[j][nowmelo] * RuiKak
  sortable = sorted(table.items(), key=lambda x:x[1], reverse=True) #確率高い順に
  afterchode = sortable[0][0] #sortable[0][0]が一番確率が高い。
  RuiKak = RuiKak * sortable[0][1]
  #print(sortable[0][0])
  henkanc = ChangeC[sortable[0][0]]#ChangeC = ['C', 'Dm', 'Em', 'F', 'G', 'Am']
  output.insert(0 , henkanc) #数字をコード名に変え、リストの0番目に挿入。順番もいれかえます

print(output)


#print(sortable[0][0])

#出力結果はメロディの順番とさかさまになっている。


cdef
[0, 1, 2, 3]
['C', 'G', 'C', 'G']


In [ ]:
#zelog未使用
user_input = input()
melo = [] #入力は ドレミファソラシド を用いる

ChangeM = ['ド', 'レ', 'ミ', 'フ', 'ソ', 'ラ', 'シ']

for i in range(len(user_input)):
  if user_input[i] == 'ァ':
    pass
  else:
    henkanm = ChangeM.index(user_input[i])
    melo.append(henkanm)

#print(melo) #user_input = ドレミ -> melo = [0,1,2]

#melo = [0,1,2,3,5,1,0] #入力
output = []

Keisan = []
for i in range(len(melo)):
  Keisan.append(melo[(len(melo) - 1) - i])
#計算する順番に合わせるため、リストの順序を逆さまにする

chode = []
table = {}
afterchode = -1 #afterchodeやtableでは、0=C 1=Dm 2=Em 3=F 4=G 5=Am
nowmelo = -1 #nowmeloでは、0＝ド,1＝レ,2＝ミ,3＝ファ,4＝ソ,5＝ラ,6＝シ
RuiKak = 1
#logRuiKak = 0

for i in range(len(Keisan)): #入力(実際は逆さまになったリスト)のメロディの個数分繰り返す
  nowmelo = Keisan[i] #いまコードを決めているメロディーをnowmeloに代入
  if i == 0:     #一番最後のコードを決める
    for j in range(6):
      table[j] = lst45[j] * lstB[j][nowmelo]
  elif i == len(Keisan) - 1:   #一番最初のコード
    for j in range(6):
      table[j] = lst01[j] * lstB[j][nowmelo] * lst12[j][afterchode] * RuiKak
  else:   #途中のコード
    for j in range(6):
      table[j] = lst12[j][afterchode] * lstB[j][nowmelo] * RuiKak
  sortable = sorted(table.items(), key=lambda x:x[1], reverse=True) #確率高い順に
  afterchode = sortable[0][0] #sortable[0][0]が一番確率が高い。
  RuiKak = RuiKak * sortable[0][1]
  #print(sortable[0][0])
  henkanc = ChangeC[sortable[0][0]]#ChangeC = ['C', 'Dm', 'Em', 'F', 'G', 'Am']
  output.insert(0 , henkanc) #数字をコード名に変え、リストの0番目に挿入。順番もいれかえます

print(output)


#print(sortable[0][0])

#出力結果はメロディの順番とさかさまになっている。


ドレミ
['C', 'G', 'C']


In [11]:
#c,d,e,f,g,a,b,c -> 0,1,2,3,4,5,6,0

del user_input
user_input = input()
melo = [] #入力は c,d,e,f,g,a,b を用いる

ChangeM = ['c', 'd', 'e', 'f', 'g', 'a', 'b']

for i in range(len(user_input)):
  henkanm = ChangeM.index(user_input[i])
  melo.append(henkanm)

print(melo)

cdefg
[0, 1, 2, 3, 4]


In [12]:
#ドレミファソラシド→01234560

del user_input
user_input = input()
melo = [] #入力は ドレミファソラシド を用いる

ChangeM = ['ド', 'レ', 'ミ', 'フ', 'ソ', 'ラ', 'シ']

for i in range(len(user_input)):
  if user_input[i] == 'ァ':
    pass
  else:
    henkanm = ChangeM.index(user_input[i])
    melo.append(henkanm)

print(melo)

ドレミファソ
[0, 1, 2, 3, 4]


In [ ]:
#lst01 始まりのコードになる確率
MC = []
lst01 = []
lst01data = []

worksheet = workbook.worksheet("最終版")
cell_list = worksheet.range("H1:H10")
for cell in cell_list:
  MC.append(cell.value)
print(MC)


for i in range(len(MC)):
  Mcqueen = ChangeC.index(MC[i])
  lst01data.append(Mcqueen)

print(lst01data)

#for j in range(len(lst01data)):       #確率を求めてリストにする
#  lst01app = lst01data.count(j) / len(lst01data)
#  lst01.append(lst01app)

for j in range(6):
  lst01app = lst01data.count(j) / len(lst01data)
  lst01.append(lst01app)

print("lst01↓")
print(lst01)

lst45 = [0.4, 0, 0, 0, 0.3, 0.3] #仮のデータ（というか、手計算でやった)
print(lst45)

#$ P_{ij} = \frac{C_{ij}}{C_{i}} $

###$ P_{ij} $:コードiからコードjへの遷移確率

###$ C_{ij} $:データベース内でコードiからjに遷移している個数

###$ C_{i} $:データベース内のコードiの個数

例:コードCからの遷移先が[Em, Em, F, G]のとき

CからEmへの遷移確率$ P_{CEm} $ = $ \frac{2}{4} $

#$ P_{ik} = \frac{C_{ik}}{C_{i}} $

###$ P_{ik} $:コードiとメロディーkの合致確率

###$ C_{ij} $:データベース内でコードiとメロディーkの組の個数

###$ C_{i} $:データベース内のコードiの個数

例:コードCとの合致メロディーが[ミ,ミ,ファ,ソ]のとき

Cからミへの遷移確率$ P_{Ce} $ = $ \frac{2}{4} $

末尾コード:$P_{i} = P_{ik}\times E_{i}$

途中のコード:$P_{i} = P_{ik}\times P_{ij}\times P$

冒頭コード:$P_{i} = P_{ik}\times P_{ij}\times B_{i}\times P$

$P_{i}$ : コードiの確率

$ P_{ik} $:コードiとメロディーkの合致確率

$ P_{ij} $:コードiからコードjへの遷移確率

$E_{i}$ : コードiが末尾コードとなる確率

$B_{i}$ : コードiが冒頭コードとなる確率

$P$ : 累積確率

In [ ]:
#計算部分のテスト
#リストAの作成

#lst12(コード) = [C,Dm,Em,F,G,Amへ移る確率]

lst01 = [0.4, 0, 0, 0.3, 0.1, 0.2] #始まりのコードになる確率[C,Dm,Em,F,G,Am]

lst12c = [0.1, 0, 0.15, 0.3, 0.3, 0.15]
lst12dm = [0.1, 0.1, 0, 0.2, 0.4, 0.2]
lst12em = [0, 0, 0.1, 0.3, 0.2, 0.4]
lst12f = [0.35, 0, 0.1, 0.1, 0.35, 0.1]
lst12g = [0.5, 0, 0.2, 0, 0.1, 0.2]
lst12am = [0.1, 0.1, 0.3, 0.4, 0, 0.1]

lst12 = [lst12c, lst12dm, lst12em, lst12f, lst12g, lst12am]

lst45 = [0.4, 0, 0, 0, 0.3, 0.3] #終わりのコードになる確率[C,Dm,Em,F,G,Am]

lstA = [lst01, lst12, lst45]

print(lstA)

[[0.4, 0, 0, 0.3, 0.1, 0.2], [[0.1, 0, 0.15, 0.3, 0.3, 0.15], [0.1, 0.1, 0, 0.2, 0.4, 0.2], [0, 0, 0.1, 0.3, 0.2, 0.4], [0.35, 0, 0.1, 0.1, 0.35, 0.1], [0.5, 0, 0.2, 0, 0.1, 0.2], [0.1, 0.1, 0.3, 0.4, 0, 0.1]], [0.4, 0, 0, 0, 0.3, 0.3]]


In [ ]:
#リストBの作成
#lstB(コード名) = [ド,レ,ミ,フ,ァ,ソ,ラ,シに合う確率]

lstBc = [0.25, 0, 0.25, 0.1, 0.25, 0.15, 0]
lstBdm = [0.1, 0.3, 0, 0.3, 0, 0.3, 0]
lstBem = [0.1, 0.15, 0.25, 0, 0.25, 0, 0.25]
lstBf = [0.25, 0.1, 0.15, 0.25, 0, 0.25, 0]
lstBg = [0, 0.25, 0.1, 0.15, 0.25, 0, 0.25]
lstBam = [0.25, 0, 0.25, 0.1, 0.15, 0.25, 0]



lstB = [lstBc, lstBdm, lstBem, lstBf, lstBg, lstBam]

print(lstB)

[[0.25, 0, 0.25, 0.1, 0.25, 0.15, 0], [0.1, 0.3, 0, 0.3, 0, 0.3, 0], [0.1, 0.15, 0.25, 0, 0.25, 0, 0.25], [0.25, 0.1, 0.15, 0.25, 0, 0.25, 0], [0, 0.25, 0.1, 0.15, 0.25, 0, 0.25], [0.25, 0, 0.25, 0.1, 0.15, 0.25, 0]]


In [ ]:
#nowmelo = Keisan[i]
#[lst12[h][nowmelo] for h in range(6)]


In [ ]:
a = [0]
b = [1, 2]
c = [3, 4, 5]

lst = [a, b, c]

circle2 = sum([len(lst[i]) for i in range(len(lst))])

print(lst)
print('--------')

print(lst[0][0])
print(lst[1][1])
print(lst[2][2])
print('--------')

print(len(lst))
print(circle2)


[[0], [1, 2], [3, 4, 5]]
--------
0
2
5
--------
3
6


In [ ]:
#③

lst2 = [[6], [7, 8], [9, 10, 11]]
lst3 = [lst, lst2]

circle31 = sum([len(lst3[i]) for i in range(len(lst3))])


print(lst3)
print('--------')
print(lst3[0][0][0])
print(lst3[1][1][1])
print(lst3[1][2][2])
print('--------')
print(len(lst3))
print(circle31)

[[[0], [1, 2], [3, 4, 5]], [[6], [7, 8], [9, 10, 11]]]
--------
0
8
11
--------
2
6
